In [1]:
## infer the probability of having a disease D given the positive test result T
## find - P(D | T )

# given probabilities
P_D = 0.01 # pior probability of having a disease 
P_not_D = 1 - P_D  # probability of not having the disease 
P_T_given_D = 0.99 # Test is positive given disease 
P_T_given_not_D = 0.05  # Test is positive given no disease

# calculate P(T)
P_T = P_T_given_D * P_D + P_T_given_not_D * P_not_D

# calulate the P(D | T) using Bayes' Theorem
P_D_given_T = (P_T_given_D * P_D) / P_T


print(f"Probability of having a disease given a positive test: {P_D_given_T:.4f}")




Probability of having a disease given a positive test: 0.1667


In [5]:
## marginal inference
import numpy as np

# joint probability table P(X, Y)
P_XY = np.array(
    [[0.1, 0.2],  # P(X=0, Y=0), P(X=0, Y=1)
    [0.3, 0.4]]  # P(X=1, Y=0), P(X=1, Y=1)
)

# Marginalization over Y to get P(X)
P_X = P_XY.sum(axis=1)

print("Marginal probability P(X):", P_X)

Marginal probability P(X): [0.3 0.7]


We want to compute:

The probability of having lung cancer given that a person coughs.
The probability of smoking given that a person has lung cancer.

Smoking (S)
Lung Cancer (L)
Coughing (C) 

P(L | C )

Step-by-Step Implementation

Define the Joint Probability Distribution (JPD).

Compute marginal probabilities by summing over irrelevant variables.

Use Bayes' Theorem for conditional probabilities

Smoking -> LungCancer

LungCancer -> Coughing

In [11]:
import numpy as np

# Define the Joint Probability Table (JPT)
# JPT[P(Smoking, LungCancer, Coughing)]
# Rows: Smoking (S), Columns: LungCancer (L), Depth: Coughing (C)
joint_prob_table = np.array([
    [[0.36, 0.04],  # P(S=0, L=0, C=0), P(S=0, L=0, C=1)
     [0.06, 0.04]], # P(S=0, L=1, C=0), P(S=0, L=1, C=1)
    [[0.24, 0.06],  # P(S=1, L=0, C=0), P(S=1, L=0, C=1)
     [0.04, 0.16]]  # P(S=1, L=1, C=0), P(S=1, L=1, C=1)
])

In [12]:
joint_prob_table[:1]

array([[[0.36, 0.04],
        [0.06, 0.04]]])

In [18]:
# 1. Compute P(L | C=1)
p_l_and_c = joint_prob_table.sum(axis=0)


In [19]:
p_l_and_c

array([[0.6, 0.1],
       [0.1, 0.2]])

P(L | C=1) = P(L and C=1) / P(C) 

In [25]:
# Normalize to compute P(L | C=1)
p__c = p_l_and_c[:, 1].sum() # P(C=1)
p_l_given_c = (p_l_and_c[:, 1] / p__c)

In [26]:
print("Print P(L | C=1)", p_l_given_c)

Print P(L | C=1) [0.33333333 0.66666667]


2. Compute P(S | L=1)

In [27]:
# marginalize over counting C
p_s_and_l = joint_prob_table.sum(axis=2)
p_s_and_l

array([[0.4, 0.1],
       [0.3, 0.2]])

P(S | L=1) = P(S and L=1) / P(L)

In [31]:
# compute P(L=1)
p_l = p_s_and_l[:, 1].sum()
p_l

np.float64(0.30000000000000004)

In [34]:
# normalize to compute P(L | C=1)
p_s_given_l = p_s_and_l[:, 1] / p_l
p_s_given_l

array([0.33333333, 0.66666667])

Basiyan Inference Sampling

Steps:
1. Define the Joint Probability Distribution
2. Monte Carlo Sampling: randonmly generate samples for (S,L,C) from P(S,L,S)
3. Extract samples where C=1
4. Approximate P(S|C=1)

In [37]:
import numpy as np

# P(S, L, C) -> Shape: (2, 2, 2) [S: Smoking, L: Lung Cancer, C: Coughing]
joint_prob_table = np.array([
    [[0.36, 0.04],  # P(S=0, L=0, C=0), P(S=0, L=0, C=1)
     [0.06, 0.04]], # P(S=0, L=1, C=0), P(S=0, L=1, C=1)
    [[0.24, 0.06],  # P(S=1, L=0, C=0), P(S=1, L=0, C=1)
     [0.04, 0.16]]  # P(S=1, L=1, C=0), P(S=1, L=1, C=1)
])


In [38]:
# flatten the JPT for sampling
flat_probs = joint_prob_table.flatten()
flat_states = np.array([
    [s, l, c] for s in [0, 1] for l in [0, 1] for c in [0, 1]
])
flat_states

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]])

In [39]:
# Monte Carlo Sampling
num_samples = 100000
samples = np.random.choice(len(flat_probs), size=num_samples, p=flat_probs)

In [47]:
sampled_states = flat_states[samples]

In [50]:
filtered_samples = sampled_states[sampled_states[:, 2] == 1]
filtered_samples

array([[0, 0, 1],
       [1, 1, 1],
       [1, 0, 1],
       ...,
       [0, 1, 1],
       [1, 0, 1],
       [1, 0, 1]])

In [51]:
# Compute P(L | C=1) using the filtered samples
num_l0 = np.sum(filtered_samples[:, 1] == 0)  # Count L=0
num_l1 = np.sum(filtered_samples[:, 1] == 1)  # Count L=1

In [52]:
# Normalize to get probabilities
p_l_given_c1 = np.array([num_l0, num_l1]) / len(filtered_samples)

In [56]:
print("P(L | C=1) [from sampling]:", p_l_given_c1)


P(L | C=1) [from sampling]: [0.33898871 0.66101129]


In [55]:

# Compare with the analytical result
p_l_and_c = joint_prob_table.sum(axis=0)  # Marginalize over S
p_c = p_l_and_c[:, 1].sum()  # P(C=1)
p_l_given_c_analytical = p_l_and_c[:, 1] / p_c
print("P(L | C=1) [analytical]:", p_l_given_c_analytical)

P(L | C=1) [analytical]: [0.33333333 0.66666667]
